# ロバストMDPと凸MDPの論文のチェック

参考：
* [Robust Reinforcement Learning with General Utility](https://openreview.net/pdf?id=8Uyfr5TcNR)

この論文結構怪しいので，検証します．
* まず第一に，Proposition 1の部分でConstrained MDPも扱える，みたいな話を書いてますが，CMDPではAssumption 2が明らかに破壊されるので間違ってる．
* Section 4.2で1/Kで収束するみたいな話が書いてますが，これも怪しいので確かめます．

**準備**

次の最適化を考えます．

$$
\min _{\pi \in \Pi} \max _{P \in U} f\left(\lambda_{\pi, P}\right)
$$

* ここで，$\lambda_{\pi, P}$はOccupancy Measureです．
* 今回は簡単のためにDirect parameterizationを使います．論文中では一般のパラメータを扱ってます．（22）式付近で言ってますが，もしかしたらSoftmax policyじゃないと無理かも．
* 論文では$s$-rectangularなUncertainty set を仮定してますが，別にいらない気がするので今回は無視します．

次の最適化を考えます．

$$
\pi_{k+1} = \pi_k - \beta_k d_k
$$

ここで，$U_k = \{P \in \arg\max_{P \in U}f(\lambda_{\pi_k, P})\}$とします．つまりMaximumを取るような$P$の集合です．
また，与えられた$d$について，勾配方向に減る量を次の用に定義します．

$$
A_k(d):=\max _{P \in U_k}\left[\nabla_\pi f\left(\lambda_{\pi_k, P}\right)^{\top} d\right], d \in B_1:=\left\{d^{\prime} \in \mathbb{R}^{d_{\Theta}}:\left\|d^{\prime}\right\| \leq 1\right\}
$$

* $P$で$\max$取ってるので，一番勾配が減らない方向ですね．
* これ無理やり論文の表記に合わせてますが，実際は$d$は$\Pi$から出ないように設定しないとダメです．まあ細かいところは一旦無視

そして，$d_k$を
$$d_k \in \arg\min_{d \in B_1} A_k(d)$$
としてます．つまり，Worst caseについて一番勾配が減るような方向を取ってます．
(28)式で，このMinimizationは

$$
d_{k, t+1} \leftarrow \operatorname{proj}_{B_1}\left[d_{k, t}-\alpha \nabla_\pi f\left(\lambda_{\pi_k, P_{k, t}}\right)\right], \text { where } P_{k, t} \in \arg \max _{P \in U_k} \nabla_\pi f\left(\lambda_{\pi_k, P}\right)^{\top} d_{k, t}
$$

* **コメント** そもそもこの$P_{k, t}$を計算する方法が特に記されてないな… 一応目的関数自体は線形なので，多分劣勾配法で収束するとは思う．試しに$f$が期待収益の時を考えてみよう．このとき，

$$
\sum_{s, a}\nabla_\pi f\left(\lambda_{\pi_k, P}\right)(s, a)d_{k, t}(s, a) = \sum_{s, a}\lambda_{\pi_k, P}(s) Q^{\pi_k}_{P}(s, a)d_{k, t}(s, a)
$$

になる．これはrectangularでもいつ最大になるのかよくわからないんじゃないかな？Occupancy measureとQの両方に$P$がかかっているけど，これを最大にする$P$はわかんないと思う．


## 収束の確認

細かいところに目をつむって，収束解析してみます．$\Gamma$を$\max_P f(...)$とします．
このとき，式163付近で，
$$
\begin{aligned}
\Gamma\left(\pi_{k+1}\right)-\Gamma\left(\pi_k\right) & \stackrel{(i)}{\leq} f\left(\lambda_{\pi_{k+1}, P_{k+1}^*}\right)-f\left(\lambda_{\pi_k, P_{k+1}^*}\right) \\
& \stackrel{(i i)}{\leq} \nabla_\pi f\left(\lambda_{\pi_k, P_{k+1}^*}\right)^{\top}\left(\pi_{k+1}-\pi_k\right)+\frac{L_{\pi, \pi}}{2}\left\|\pi_{k+1}-\pi_k\right\|^2 \\
& \stackrel{(i i i)}{=} \beta_k \nabla_\pi f\left(\lambda_{\pi_k, P_{k+1}^*}\right)^{\top} d_k+\frac{L_{\pi, \pi}}{2} \beta_k^2 \\
& \stackrel{(i v)}{\leq} \beta_k A_k\left(d_k\right)+\frac{L_{\pi, \pi}}{2} \beta_k^2 \\
&\leq \text{上からGradient dominanceで抑える}
\end{aligned}
$$

みたいなことをしていますが，途中で$A_k\left(d_k\right)$を使って抑えています．これは正しいですが，現実的じゃなさそう．
* 途中過程で$P_{k+1}^*$がでるので，なんとかしてこれを更新中の勾配に出さないといけないわけですね．
* もちろん$P_{k+1}^*$は$\pi_{k+1}$に依存してるので，計算できません．そこで，論文ではそれを更に上から抑える量で抑えていますが，上で見たように，$d_k$の計算ができるかは怪しいです．
